In [8]:
!pip install xlsx2csv

     ---------------------------------------- 0.0/227.1 kB ? eta -:--:--
     ---------------------------------------- 0.0/227.1 kB ? eta -:--:--
     - -------------------------------------- 10.2/227.1 kB ? eta -:--:--
     ----- ------------------------------- 30.7/227.1 kB 262.6 kB/s eta 0:00:01
     ------ ------------------------------ 41.0/227.1 kB 279.3 kB/s eta 0:00:01
     ----------------- ------------------ 112.6/227.1 kB 544.7 kB/s eta 0:00:01
     ------------------------------------ 227.1/227.1 kB 990.3 kB/s eta 0:00:00
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Created wheel for xlsx2csv: filename=xlsx2csv-0.8.2-py3-none-any.whl size=14229 sha256=b95bd14234bf2e69996deabae178ee1c39991632f

In [6]:
import polars as pl

In [39]:
df = pl.read_excel("tabel_3_1.xlsx")
df

pad1,pad2,pad3,pad4
i64,i64,i64,i64
933,937,938,935
897,898,915,913
840,900,900,930
900,905,902,900
879,852,873,871
903,890,892,908


In [25]:
def buat_kelompok(df):
    n = df.shape[0]
    grup = []
    num_kontrol = 1

    for i in range(int(n/2)):
        for j in range(2):        
            grup.append(f"{num_kontrol+i}-{num_kontrol+i+1}")
            num_kontrol += j
    return grup


In [47]:
grup = buat_kelompok(df)
df = df.with_columns(pl.Series(grup).alias('grup'))

In [106]:
n_pad = 4
n = df.shape[0]
genap = [x-1 for x in range(1, n + 1) if x % 2 != 0]
ganjil = [x-1 for x in range(1, n + 1) if x % 2 == 0]

a = []
for i in genap:
    for j in range(n_pad):
        a.append(df[i,j])

b = []
for i in ganjil:
    for j in range(n_pad):
        b.append(df[i,j])

df_kelompok = df.select([pl.col("grup").repeat_by(2).explode()])
df_kelompok = df_kelompok.with_columns([
    pl.Series(a).alias("a"),
    pl.Series(b).alias("b")
])
df_kelompok = df_kelompok.with_columns(pl.mean_horizontal(pl.col("a", "b")).alias("mean"))
df_kelompok.with_columns(R=(pl.col("a")-pl.col("b")).abs())

grup,a,b,mean,R
str,i64,i64,f64,i64
"""1-2""",933,897,915.0,36
"""1-2""",937,898,917.5,39
"""1-2""",938,915,926.5,23
"""1-2""",935,913,924.0,22
"""3-4""",840,900,870.0,60
…,…,…,…,…
"""3-4""",930,900,915.0,30
"""5-6""",879,903,891.0,24
"""5-6""",852,890,871.0,38
